# Grain Process

In [1]:
import yt.units as u
import numpy as np
atoms = ['H', 'C', 'O', 'N']
mass = {'H': 1.0079, 'C': 12.0107, 'O': 15.9994, 'N': 14.0067}

## Grain model

In [2]:
r = 1000 * u.angstrom
V = 4 / 3 * np.pi * r**3
rho = 3 * u.gram / u.cm**3
Ns = 1e6
G_M_ratio = 100
nH = 2e4 / u.cm**3
T = 10 * u.Kelvin
nd = nH * u.mass_hydrogen / G_M_ratio / rho / V

In [3]:
nH/nd

750798001716.7773 dimensionless

In [15]:
(nd*np.pi*r**2/nH/(1e-27*u.m**2)).in_cgs()

41.843380595129666 dimensionless

In [16]:
(rho*V/mass['C']/u.mass_hydrogen).in_cgs()

625107613.8083353 dimensionless

In [14]:
rho*V/mass['H']/u.mass_hydrogen

7449131875.352488 dimensionless

In [4]:
def grain_spe(spe):
    if spe == ' ' or len(spe) == 0:
        return ('')
    t = ''
    grain_spe = ''
    for i in spe:
        if i.isdigit():
            grain_spe += (t + '_dust') * (int(i) - 1)
        else:
            t = i
            grain_spe += (t + '_dust')
    return(grain_spe)

## Accretion

$$R_{\mathrm{acc}}(i)=\overline{\sigma_{d}n_{d}}\langle v(i)\rangle n(i) = \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)$$

$$k_i= \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)=6.0557\sqrt{\frac{T\ (\text{K})}{\mu}}\times10^{-14}\text{ s}^{-1}$$

In [41]:
def mu(spe):
    t = ''
    m = 0
    for i in spe:
        if i.isdigit():
            m += mass[t] * (int(i) - 1)
        else:
            t = i
            m += mass[t]
    return(m)

def acc(spe, label=0):
    react = '{},{},{},NONE,NONE,6.0557e-14*sqrt(Tgas/{:.4f})'.format(
        label, spe, grain_spe(spe), mu(spe))
    return(react)

In [42]:
LIST_complex = [
    'H', 'C', 'N', 'O', 'H2', 'C2', 'N2', 'O2', 'CH', 'NH', 'OH', 'CN', 'CO',
    'NO', 'O3', 'C2H', 'N2H', 'O2H', 'CH2', 'NH2', 'H2O', 'C2N', 'CO2', 'HCN',
    'HNC', 'HCO', 'HOC', 'HNO', 'OCN', 'CH3', 'NH3', 'C2H2', 'N2H2', 'H2O2',
    'H2CO', 'CHOH', 'HCCN', 'CH4', 'C2H3', 'CH2OH', 'CH2CN', 'C2H4', 'CH3OH',
    'CH3CN', 'C2H5', 'C2H6'
]
for i, spe in enumerate(LIST_complex, 4432):
    print(acc(spe, label=i))

4432,H,H_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/1.0079)
4433,C,C_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/12.0107)
4434,N,N_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/14.0067)
4435,O,O_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/15.9994)
4436,H2,H_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/2.0158)
4437,C2,C_dustC_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/24.0214)
4438,N2,N_dustN_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/28.0134)
4439,O2,O_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/31.9988)
4440,CH,C_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/13.0186)
4441,NH,N_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/15.0146)
4442,OH,O_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/17.0073)
4443,CN,C_dustN_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/26.0174)
4444,CO,C_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/28.0101)
4445,NO,N_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/30.0061)
4446,O3,O_dustO_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/47.9982)
4447,C2H,C_dustC_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/25.0293)
4448,N2H,N_dustN_dustH_dust,NONE,NONE,6.0

## Evaporation

### Thermal Evaporation

$$k_{\mathrm{eva, thermal}}(i)=\nu_{i} \exp \left[-E_{D} / kT\right]$$

$$\begin{aligned} \nu_{i} &=\left(2 \rho_{s} E_{D} / \pi^{2} m_{i}\right)^{1 / 2} \\ &=2.4 \times 10^{12} \mathrm{Hz}\left(\frac{\rho_{s}}{10^{15} \mathrm{cm}^{-2}}\right)^{1 / 2}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2} \end{aligned}$$

In [7]:
rhos = Ns/4/np.pi/r**2
np.sqrt(rhos.in_cgs()/1e15*u.cm**2)

0.8920620580763855 dimensionless

As a result,

$$k_{\mathrm{eva, thermal}}(i)=2.14 \times 10^{12} \mathrm{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2} \exp \left[-E_{D} / T\right]$$

### Cosmic Ray Desorption

$$k_{\mathrm{eva}, \mathrm{cosmic}}(i)=\zeta(70 \mathrm{K}) \cdot \nu_{i} \exp \left[-E_{D}/(70 \mathrm{K})\right]$$

$\zeta(70\text{K})\sim3.16 \times 10^{-19}$ (Hasegawa & Herbst, 1993), thus

$$k_{\mathrm{eva}, \mathrm{cosmic}}(i)=6.76\times10^{-7}\text{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2}\exp\left[-E_{D} / (70\text{K})\right]$$

### Photodesorption

- R. Visser et al. 2011

Laboratory experiments on the photodesorption of H2O, CO and CO2 all produce a yield of $Y\approx10^{-3}$ molecules per grain per incident UV photon (Öberg et al. 2007, 2009a), while the yield for N2 is an order of magnitude lower (Öberg et al. 2009b). For all other ice species in our network, whose photodesorption yields have not yet been deter- mined experimentally or theoretically, the yield is set to $10^{−3}$. The photodesorption rate is

$$
R_{\mathrm{phdes}}(X)=\pi a^{2} n_d f(X) Y(X) F_{0} \exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)
$$

where

$$
f(X)=\frac{n_{\mathrm{s}}(X)}{\max \left(n_{\mathrm{ice}}, N_{s} n_{d}\right)}
$$

Here $n_{d}=2.66\times10^{-8}$ cm$^{-3}$, $N_s=10^6$, thus only when $n_{\mathrm{ice}}>2.66\times10^{-2}$ cm$^{-3}$ do we take $f(X)$ as ${n_{\mathrm{s}}(X)}/{n_{\mathrm{ice}}}$ and in other cases

$$
k_{\mathrm{eva,UV}}(i)\approx10^{-9}\pi a^{2}  F_{0} \exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)
$$

Photodesorption occurs even in strongly shielded regions because of cosmic-ray-induced photons, which is accounted for by setting a lower limit of $10^4$ cm$^{−2}$ s$^{−1}$ to the product $F_{0} \exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)$.

In [8]:
(1e-3*np.pi*r**2).in_cgs()

3.1415926535897937e-13 cm**2

Thus,

$$
k_{\mathrm{eva,UV}}(i)=3.14\times 10^{-19} \text{Hz}\cdot \frac{F_{0}}{\text{cm}^{-2}\text{ s}^{-1}} \exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)
$$

The UV background we use is taken from Draine 1978:

$$F(E)=\left[1.658 \times 10^{6}(E / \mathrm{eV})-2.152 \times 10^{5}(E / \mathrm{eV})^{2}+6.919 \times 10^{3}(E / \mathrm{eV})^{3}\right] \text { photons } \mathrm{cm}^{-2} \mathrm{s}^{-1}\ \mathrm{sr}^{-1}\ \mathrm{eV}^{-1}$$

Between 5 eV and 13.6 eV,

$$F=2.42 \times 10^{8} \text { photons } \mathrm{cm}^{-2} \mathrm{s}^{-1}$$

Thus,

$$
k_{\mathrm{eva,UV}}(i)=7.59\times 10^{-11} \text{Hz}\exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)
$$

In [9]:
(1.658e6/2*(13.6**2-5**2)-2.152e5/3*(13.6**3-5**3)+6.919e3/4*(13.6**4-5**4))*4*np.pi

241594323.46166772

In [43]:
def eva(spe, Eads, label=0):
    kth = 2.14e12 / u.s * np.sqrt(Eads / 350 / u.Kelvin) / np.sqrt(
        mu(spe) / mu('H'))
    kcos = 6.76e-7 / u.s * np.sqrt(Eads / 350 / u.Kelvin) / np.sqrt(
        mu(spe) / mu('H')) * np.exp(-Eads / 70 / u.Kelvin)
    kuv = 7.59e-11 / u.s
    
    react = '{},{},{},NONE,NONE,{:e}*exp(-{:g}/Tgas)+{:e}+{}*exp(-user_Av)'.format(
        label, grain_spe(spe), spe, kth, Eads, kcos, kuv)
    return (react)

In [44]:
calorie = 41858000 * u.erg
kcal = 1e3 * calorie
3.8 * kcal / 6.02e23 / u.boltzmann_constant/1910

1.001956788330078 K

In [48]:
calorie = 41858000 * u.erg
kcal = 1e3 * calorie
Eads = {}
Eads['H'] = 350 * u.Kelvin
Eads['C'] = 800 * u.Kelvin
Eads['N'] = 800 * u.Kelvin
Eads['O'] = 800 * u.Kelvin
Eads['H2'] = 450 * u.Kelvin
Eads['C2'] = 1210 * u.Kelvin
Eads['N2'] = 1210 * u.Kelvin
Eads['O2'] = 1210 * u.Kelvin
Eads['CH'] = 654 * u.Kelvin
Eads['NH'] = 604 * u.Kelvin
Eads['OH'] = 1260 * u.Kelvin
Eads['CN'] = 1510 * u.Kelvin
Eads['CO'] = 1210 * u.Kelvin
Eads['NO'] = 1210 * u.Kelvin
Eads['O3'] = 2520 * u.Kelvin
Eads['C2H'] = 1460 * u.Kelvin
Eads['N2H'] = 1510 * u.Kelvin
Eads['O2H'] = 1510 * u.Kelvin
Eads['CH2'] = 956 * u.Kelvin
Eads['NH2'] = 856 * u.Kelvin
Eads['H2O'] = 1860 * u.Kelvin
Eads['C2N'] = 2010 * u.Kelvin
Eads['CO2'] = 2500 * u.Kelvin
Eads['HCN'] = 1760 * u.Kelvin
Eads['HNC'] = 1510 * u.Kelvin
Eads['HCO'] = 1510 * u.Kelvin
Eads['HOC'] = 1760 * u.Kelvin
Eads['HNO'] = 1510 * u.Kelvin
Eads['OCN'] = 2010 * u.Kelvin
Eads['CH3'] = 1160 * u.Kelvin
Eads['NH3'] = 1110 * u.Kelvin
Eads['C2H2'] = 1610 * u.Kelvin
Eads['N2H2'] = 1210 * u.Kelvin
Eads['H2O2'] = 2520 * u.Kelvin
Eads['H2CO'] = 1760 * u.Kelvin
Eads['CHOH'] = 1910 * u.Kelvin
Eads['HCCN'] = 2270 * u.Kelvin
Eads['CH4'] = 1360 * u.Kelvin
Eads['C2H3'] = 1760 * u.Kelvin
Eads['CH2OH'] = 2170 * u.Kelvin
Eads['CH2CN'] = 2470 * u.Kelvin
Eads['C2H4'] = 2010 * u.Kelvin
Eads['CH3OH'] = 2060 * u.Kelvin
Eads['CH3CN'] = 2270 * u.Kelvin
Eads['C2H5'] = 2110 * u.Kelvin
Eads['C2H6'] = 2320 * u.Kelvin
print(len(Eads))
for item, i in enumerate(Eads.keys()):
    print(i)
    print(Eads[i])
    kth = 2.14e12 / u.s * np.sqrt(Eads[i] / 350 / u.Kelvin) / np.sqrt(
        mu(i) / mu('H')) * np.exp(-Eads[i] / 10 / u.Kelvin)
    kcos = 6.76e-7 / u.s * np.sqrt(Eads[i] / 350 / u.Kelvin) / np.sqrt(
        mu(i) / mu('H')) * np.exp(-Eads[i] / 70 / u.Kelvin)
    kuv = 7.59e-11
    print('{:e} {:e} {:e}'.format(kth, kcos, kuv))

46
H
350.0 K
1.349295e-03 4.554852e-09 7.590000e-11
C
800.0 K
1.691573e-23 3.221193e-12 7.590000e-11
N
800.0 K
1.566416e-23 2.982861e-12 7.590000e-11
O
800.0 K
1.465626e-23 2.790931e-12 7.590000e-11
H2
450.0 K
4.911559e-08 8.752071e-10 7.590000e-11
C2
1210.0 K
2.299057e-41 8.009854e-15 7.590000e-11
N2
1210.0 K
2.128952e-41 7.417215e-15 7.590000e-11
O2
1210.0 K
1.991966e-41 6.939959e-15 7.590000e-11
CH
654.0 K
3.219112e-17 2.252044e-11 7.590000e-11
NH
604.0 K
4.275268e-15 4.116632e-11 7.590000e-11
OH
1260.0 K
1.878674e-43 4.755414e-15 7.590000e-11
CN
1510.0 K
2.309289e-54 1.183384e-16 7.590000e-11
CO
1210.0 K
2.129078e-41 7.417651e-15 7.590000e-11
NO
1210.0 K
2.057046e-41 7.166696e-15 7.590000e-11
O3
2520.0 K
3.005847e-98 6.096883e-23 7.590000e-11
C2H
1460.0 K
3.435945e-52 2.423435e-16 7.590000e-11
N2H
1510.0 K
2.186511e-54 1.120467e-16 7.590000e-11
O2H
1510.0 K
2.050261e-54 1.050646e-16 7.590000e-11
CH2
956.0 K
2.872700e-30 3.508762e-13 7.590000e-11
NH2
856.0 K
5.602122e-26 1.296260e-1

In [46]:
for item, i in enumerate(Eads.keys(), 4478):
    print(eva(i, Eads[i], label=item))

4478,H_dust,H,NONE,NONE,2.140000e+12*exp(-350/Tgas)+4.554852e-09+7.59e-11*exp(-user_Av)
4479,C_dust,C,NONE,NONE,9.372367e+11*exp(-800/Tgas)+3.221193e-12+7.59e-11*exp(-user_Av)
4480,N_dust,N,NONE,NONE,8.678917e+11*exp(-800/Tgas)+2.982861e-12+7.59e-11*exp(-user_Av)
4481,O_dust,O,NONE,NONE,8.120478e+11*exp(-800/Tgas)+2.790931e-12+7.59e-11*exp(-user_Av)
4482,H_dustH_dust,H2,NONE,NONE,1.715817e+12*exp(-450/Tgas)+8.752071e-10+7.59e-11*exp(-user_Av)
4483,C_dustC_dust,C2,NONE,NONE,8.150458e+11*exp(-1210/Tgas)+8.009854e-15+7.59e-11*exp(-user_Av)
4484,N_dustN_dust,N2,NONE,NONE,7.547415e+11*exp(-1210/Tgas)+7.417215e-15+7.59e-11*exp(-user_Av)
4485,O_dustO_dust,O2,NONE,NONE,7.061782e+11*exp(-1210/Tgas)+6.939959e-15+7.59e-11*exp(-user_Av)
4486,C_dustH_dust,CH,NONE,NONE,8.139453e+11*exp(-654/Tgas)+2.252044e-11+7.59e-11*exp(-user_Av)
4487,N_dustH_dust,NH,NONE,NONE,7.283667e+11*exp(-604/Tgas)+4.116632e-11+7.59e-11*exp(-user_Av)
4488,O_dustH_dust,OH,NONE,NONE,9.884531e+11*exp(-1260/Tgas)+4.755414e-15+7.

## Surface reactions

$$k_{ij}=\kappa_{ij}(R_{\mathrm{diff},i}+R_{\mathrm{diff},j})/n_d$$

$$\kappa_{ij}=\exp[-2(1 \overset{\circ}{\text{A}}/\hbar)(2\mu E_a)^{1/2}]$$

$$R_{\mathrm{diff},i}=N_s^{-1}\nu_o\exp[-E_b/T]\approx N_s^{-1}\nu_i\exp[-0.3E_D/T]=2.14 \times 10^{6} \mathrm{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2}\exp[-0.3E_D/T]$$

except for $\mathrm{H}$ and $\mathrm{H}_2$, of which

$$R_{\mathrm{diff},i}=\nu_{i} \exp \left[-(2 \overset{\circ}{\text{A}} / \hbar)\left(2 m E_{b}\right)^{1 / 2}\right]=2.14 \times 10^{12} \mathrm{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2}\exp \left[-(2 \overset{\circ}{\text{A}} / \hbar)\left(2 m E_{b}\right)^{1 / 2}\right]$$

In [23]:
import pandas as pd

In [24]:
def Rdiff(spe, T=10 * u.K):
    if spe in ['H2', 'H']:
        return (2.14e6 / u.s * np.sqrt(Eads[spe] / 350 / u.K) /
                np.sqrt(mu(spe) / mu('H')) * np.exp(
                    (-(2 * 0.3 * Eads[spe] * mu(spe) / mu('H') *
                       u.mass_hydrogen * u.boltzmann_constant)**(1 / 2) *
                     (2 * u.angstrom / u.hbar)).in_cgs()))
    else:
        #2.14e6 - Ns=1e6
        #3.6e6 - ns=1.4e15
        return (2.14e6 / u.s * np.sqrt(Eads[spe] / 350 / u.K) /
                np.sqrt(mu(spe) / mu('H')) * np.exp(-0.3 * Eads[spe] / T))

In [25]:
Rdiff('H'), Rdiff('H2'), Rdiff('C'), Rdiff('N')

(32810.63869021714 1/s,
 2113.2928673734577 1/s,
 3.538194816255683e-05 1/s,
 3.276408078983327e-05 1/s)

In [26]:
def kappa(spei, spej, Ea):
    muij = mu(spei) * mu(spej) / (mu(spei) + mu(spej)) * (u.mass_hydrogen /
                                                          mu('H'))
    return np.exp((-2 * (u.angstrom / u.hbar) * (2 * muij * Ea * u.boltzmann_constant)**(1 / 2)).in_cgs())

In [27]:
nd.in_cgs()

2.663832342956152e-08 cm**(-3)

In [28]:
pd.read_csv('../../Notes/grain_reaction.csv').fillna('')

,R1,R2,P1,P2,k_ij,Ea
0,H,H,H2,,3.880000e+12,0
1,H,C,CH,,1.940000e+12,0
2,H,N,NH,,1.940000e+12,0
3,H,O,OH,,1.940000e+12,0
4,H,CH,CH2,,1.940000e+12,0
5,H,NH,NH2,,1.940000e+12,0
6,H,OH,H2O,,1.940000e+12,0
7,H,C2,C2H,,1.940000e+12,0
8,H,CN,HCN,,1.940000e+12,0
9,H,CO,HOC,,2.570000e+06,1000


In [35]:
reaction = pd.read_csv('../../Notes/grain_reaction.csv').fillna('')
for i in reaction.index:
    rec = reaction.loc[i]
    print(rec)
    k = kappa(rec['R1'], rec['R2'],
              rec['Ea'] * u.K) * (Rdiff(rec['R1']) + Rdiff(rec['R2'])) / nd
    print('{:e}, {:e}'.format(k.in_cgs(), k.in_cgs()/rec['k_ij']))

R1             H
R2             H
P1            H2
P2              
k_ij    3.88e+12
Ea             0
Name: 0, dtype: object
2.463416e+12, 6.349011e-01
R1             H
R2             C
P1            CH
P2              
k_ij    1.94e+12
Ea             0
Name: 1, dtype: object
1.231708e+12, 6.349011e-01
R1             H
R2             N
P1            NH
P2              
k_ij    1.94e+12
Ea             0
Name: 2, dtype: object
1.231708e+12, 6.349011e-01
R1             H
R2             O
P1            OH
P2              
k_ij    1.94e+12
Ea             0
Name: 3, dtype: object
1.231708e+12, 6.349011e-01
R1             H
R2            CH
P1           CH2
P2              
k_ij    1.94e+12
Ea             0
Name: 4, dtype: object
1.231708e+12, 6.349011e-01
R1             H
R2            NH
P1           NH2
P2              
k_ij    1.94e+12
Ea             0
Name: 5, dtype: object
1.231708e+12, 6.349013e-01
R1             H
R2            OH
P1           H2O
P2              
k_ij    1.94e+12
Ea 

1.150849e+03, 7.148128e-01
R1          CH
R2          CH
P1        C2H2
P2            
k_ij    263000
Ea           0
Name: 66, dtype: object
1.841877e+05, 7.003334e-01
R1          CH
R2          OH
P1        CHOH
P2            
k_ij    131000
Ea           0
Name: 67, dtype: object
9.209384e+04, 7.030064e-01
R1          CH
R2         HNO
P1          NO
P2         CH2
k_ij    131000
Ea           0
Name: 68, dtype: object
9.209384e+04, 7.030064e-01
R1          CH
R2         CH3
P1        C2H4
P2            
k_ij    131000
Ea           0
Name: 69, dtype: object
9.209387e+04, 7.030066e-01
R1           OH
R2           OH
P1         H2O2
P2             
k_ij    0.00397
Ea            0
Name: 70, dtype: object
2.845429e-03, 7.167327e-01
R1         OH
R2        CH2
P1      CH2OH
P2           
k_ij     17.1
Ea          0
Name: 71, dtype: object
1.246866e+01, 7.291612e-01


In [47]:
for item, i in enumerate(reaction.index, 4524):
    rec = reaction.loc[i]
    ki = (kappa(rec['R1'], rec['R2'], rec['Ea'] * u.K) * Rdiff(rec['R1']) / nd).in_cgs()
    kj = (kappa(rec['R1'], rec['R2'], rec['Ea'] * u.K) * Rdiff(rec['R2']) / nd).in_cgs()
    if not rec['R1'] in ['H', 'H2']:
        ki /= np.exp(-0.3 * Eads[rec['R1']] / 10 / u.K)
        ki_string = '{:e}*exp(-0.3*{}/Tgas)'.format(ki, Eads[rec['R1']])
    else:
        ki_string = '{:e}'.format(ki)
    if not rec['R2'] in ['H', 'H2']:
        kj /= np.exp(-0.3 * Eads[rec['R2']] / 10 / u.K)
        kj_string = '{:e}*exp(-0.3*{:g}/Tgas)'.format(kj, Eads[rec['R2']])
    else:
        kj_string = '{:e}'.format(kj)
    print('{},{},{},{},{},NONE,NONE,{}+{}'.format(item, grain_spe(rec['R1']),
                                                  grain_spe(rec['R2']),
                                                  grain_spe(rec['P1']),
                                                  grain_spe(rec['P2']), ki_string,
                                                  kj_string))

4524,H_dust,H_dust,H_dustH_dust,,NONE,NONE,1.231708e+12+1.231708e+12
4525,H_dust,C_dust,C_dustH_dust,,NONE,NONE,1.231708e+12+3.518377e+13*exp(-0.3*800/Tgas)
4526,H_dust,N_dust,N_dustH_dust,,NONE,NONE,1.231708e+12+3.258057e+13*exp(-0.3*800/Tgas)
4527,H_dust,O_dust,O_dustH_dust,,NONE,NONE,1.231708e+12+3.048420e+13*exp(-0.3*800/Tgas)
4528,H_dust,C_dustH_dust,C_dustH_dustH_dust,,NONE,NONE,1.231708e+12+3.055542e+13*exp(-0.3*654/Tgas)
4529,H_dust,N_dustH_dust,N_dustH_dustH_dust,,NONE,NONE,1.231708e+12+2.734281e+13*exp(-0.3*604/Tgas)
4530,H_dust,O_dustH_dust,H_dustH_dustO_dust,,NONE,NONE,1.231708e+12+3.710643e+13*exp(-0.3*1260/Tgas)
4531,H_dust,C_dustC_dust,C_dustC_dustH_dust,,NONE,NONE,1.231708e+12+3.059674e+13*exp(-0.3*1210/Tgas)
4532,H_dust,C_dustN_dust,H_dustC_dustN_dust,,NONE,NONE,1.231708e+12+3.284264e+13*exp(-0.3*1510/Tgas)
4533,H_dust,C_dustO_dust,H_dustO_dustC_dust,,NONE,NONE,3.883765e+06+8.934333e+07*exp(-0.3*1210/Tgas)
4534,H_dust,C_dustO_dust,H_dustC_dustO_dust,,NONE,NONE,3.883765